# Get started

One way to think about LaminDB is as queryable version of git for data.

```{note}

Similar to how you work with _repositories_ in git, you work with _instances_ in LaminDB.

A LaminDB instance is a data warehouse that makes storage of large-scale data queryable, managing both the storage location for data and a SQL database for querying it.

- Storage locations can be either local directories or cloud object stores (S3, GCP)
- Databases can be either sqlite or Postgres

```

To get started, we'll simply manage a local directory with sqlite!

In [ ]:
from nbproject import header
from lamindb.do import ingest
import scanpy as sc

header()

## Set up your LaminDB instance

```{important}

As a first time user, you need to sign up your email in the cloud. This ensure you have an unambiguous user identity across instances and enables collaboration in the cloud, similar to GitHub, HuggingFace, Benchling, and other platforms.

You sign up on the command line via `lndb signup --email <your-email>`, for example:

    $lndb signup --email "raspbear@gmx.de"

    Please *confirm* the sign-up email. After that, proceed to `lndb init`!

    Generated login secret: O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR.
    Email & secret persist in: /Users/falexwolf/.lndb/lndb.env.
    Going forward, credentials are auto-loaded. In case of loss, you can always recover your secret via email.

For a detailed walk-through of different configurations steps, see this guide: {doc}`../guides/configure`.
```

```{note}

After confirming the sign up email you can right away initialize your first instance via:

    lndb init --storage mydata
```

This tutorial assumes that user `raspbear@gmx.de` completed the cloud sign up,[^ci] but has not yet logged into LaminDB in the present compute environment.

[^ci]: This is for pure CI reasons -- all code in this tutorial is tested. Simulating a first-time sign-up at every CI run is neither manageable nor useful.

In [ ]:
!lndb login --email "raspbear@gmx.de" --secret "O6A5bqHzhERTMpGHXxEdbsTrEIt4a3Dy3AWdrWQR"

We can now initialize our first instance:

In [ ]:
!lndb init --storage mydata

## Ingest data

For the sake of demonstration, let's get some piece of data from somewhere and write it to a file:

In [ ]:
sc.datasets.pbmc68k_reduced().write("scanpy_pbmc68k_reduced.h5ad")

Add the file to the to-be-ingested list

In [ ]:
ingest.add("scanpy_pbmc68k_reduced.h5ad")

Check the to-be-ingested list with assigned Dobject ids

In [ ]:
ingest.status

Let's ingest the file via:

In [ ]:
# Setting "integrity" to False and "i_confirm_i_saved" to True is both related to the CI env
# It's not needed in production in Jupyter Lab
ingest.commit(integrity=False, i_confirm_i_saved=True)